# Investigate query runtime, join indexes, and complexity estimation

By evaluating all join indexes

In [1]:
import json
import random
import time
import bz2

import numpy
import pandas
import py2neo

import hetio.neo4j
import hetio.readwrite

## Set up and execute neo4j queries

In [2]:
args = {
    'n_metapaths': 50,
    'port': 7499,
    'n_positives': 200
}

In [3]:
# Load metagraph for the hetnet
commit = '0f2ef740197dd2767cb0de80f57d9f47e2e91c7a'
url = 'https://github.com/dhimmel/integrate/raw/{}/data/metagraph.json'.format(commit)
metagraph = hetio.readwrite.read_metagraph(url)

In [4]:
# Read metapath information
with open('../../all-features/data/metapaths.json') as fp:
    metapaths = json.load(fp)

# Randomly select a subset of metapaths
random.seed(0)
metapaths = random.sample(metapaths, args['n_metapaths'])
abbrev_to_item = {item['abbreviation']: item for item in metapaths}
metapaths = [metagraph.metapath_from_abbrev(metapath['abbreviation']) for metapath in metapaths]

Note: it would be helpful to add no join hint: see what happens when you let neo4j decide.

In [5]:
# Create a dataframe of estimated complexity
rows = list()
for abbrev, item in abbrev_to_item.items():
    for i, complexity, in enumerate(item['join_complexities']):
        row = abbrev, i, complexity, i == item['optimal_join_index'], i == item['midpoint_index']
        rows.append(row)

index_df = pandas.DataFrame(rows, columns=['metapath', 'join_index', 'complexity', 'optimal_index', 'midpoint_index'])
index_df.head(3)

,metapath,join_index,complexity,optimal_index,midpoint_index
0,CdGcGr>GuD,0,4.812894,False,False
1,CdGcGr>GuD,1,4.573193,False,False
2,CdGcGr>GuD,2,3.812083,False,True


In [6]:
# Create a dataframe of queries
rows = list()
for metapath in metapaths:
    for i in range(len(metapath) + 1):
        query = hetio.neo4j.construct_dwpc_query(metapath, join_hint=i)
        rows.append((str(metapath), query, i))
query_df = pandas.DataFrame(rows, columns=['metapath', 'query', 'join_index'])

In [7]:
# Connect to neo4j server
uri = 'http://localhost:{}/db/data/'.format(args['port'])
neo = py2neo.Graph(uri)

In [8]:
# Retreive network positives
query ='''
MATCH (c:Compound)-[:TREATS_CtD]->(d:Disease)
RETURN d.name AS disease, c.name AS compound
'''
results = neo.cypher.execute(query)
pair_df = pandas.DataFrame(results.records, columns=results.columns)
pair_df = pair_df.sample(args['n_positives'], random_state=0)

In [9]:
# Merge queries and pairs
pair_df['dummy'] = 1
query_df['dummy'] = 1
query_df = pair_df.merge(query_df)
del query_df['dummy']
query_df.head(2)

,disease,compound,metapath,query,join_index
0,epilepsy syndrome,Carbamazepine,CrCdGuCpD,MATCH path = (n0:Compound)-[:RESEMBLES_CrC]-(n...,0
1,epilepsy syndrome,Carbamazepine,CrCdGuCpD,MATCH path = (n0:Compound)-[:RESEMBLES_CrC]-(n...,1


In [10]:
# Number of queries
len(query_df)

48800

In [11]:
# Perform queries
rows = list()
for row in query_df.itertuples():
    start = time.time()
    result = neo.cypher.execute(row.query, source=row.compound, target=row.disease, w=0.4)
    seconds = time.time() - start
    row = list(row)[1:] + list(result.records[0]) + [seconds]
    rows.append(row)
    print('Completed {:.4%}'.format(len(rows) / len(query_df)), end='\r')

result_df = pandas.DataFrame(rows, columns= list(query_df.columns) + list(result.columns) + ['seconds'])

In [12]:
len(rows)

48800

In [14]:
result_df.head(2)

,disease,compound,metapath,query,join_index,PC,DWPC,seconds
0,epilepsy syndrome,Carbamazepine,CrCdGuCpD,MATCH path = (n0:Compound)-[:RESEMBLES_CrC]-(n...,0,0,0.0,0.006906
1,epilepsy syndrome,Carbamazepine,CrCdGuCpD,MATCH path = (n0:Compound)-[:RESEMBLES_CrC]-(n...,1,0,0.0,0.006415


## Analyze query results

In [15]:
complexity_df = result_df.merge(index_df).sort_values(['disease', 'compound', 'metapath', 'join_index'])
del complexity_df['query']
complexity_df.seconds = complexity_df.seconds.astype(float)
complexity_df['log_seconds'] = numpy.log10(complexity_df.seconds)
complexity_df.head(3)

,disease,compound,metapath,join_index,PC,DWPC,seconds,complexity,optimal_index,midpoint_index,log_seconds
21944,Alzheimer's disease,Galantamine,CbG<rGr>GaD,0,358,0.001389,0.150434,4.182732,False,False,-0.822655
22144,Alzheimer's disease,Galantamine,CbG<rGr>GaD,1,358,0.001389,8.675076,4.440559,False,False,0.938273
22344,Alzheimer's disease,Galantamine,CbG<rGr>GaD,2,358,0.001389,0.046207,3.236234,False,True,-1.335290


In [24]:
with bz2.open('data/query-results.tsv.bz2', 'wt') as write_file:
    complexity_df.to_csv(write_file, sep='\t', index=False, float_format='%.5g')

In [17]:
len(complexity_df)

48800

In [18]:
# Summarize by query
def summary(df):
    row = pandas.Series()
    
    df = df.sort_values('seconds')
    row['optimal_is_best'] = df.optimal_index.iloc[0]
    row['midpoint_is_best'] = df.midpoint_index.iloc[0]
    
    df = df.sort_values('complexity')
    row['rank_of_fastest'] = 1 + numpy.argmin(list(df.seconds))
    min_seconds = df.seconds.min()
    row['min_seconds'] = min_seconds
    row['delay_of_optimal'] = df.loc[df.optimal_index, 'seconds'].iloc[0] - min_seconds
    row['delay_of_midpoint'] = df.loc[df.midpoint_index, 'seconds'].iloc[0] - min_seconds

    return row

query_summary_df = complexity_df.groupby(['disease', 'compound', 'metapath']).apply(summary).reset_index()
for column in 'optimal_is_best', 'midpoint_is_best', 'rank_of_fastest':
    query_summary_df[column] = query_summary_df[column].astype(int)
query_summary_df.head()

,disease,compound,metapath,optimal_is_best,midpoint_is_best,rank_of_fastest,min_seconds,delay_of_optimal,delay_of_midpoint
0,Alzheimer's disease,Galantamine,CbG<rGr>GaD,0,1,2,0.046207,0.104974,0.000000
1,Alzheimer's disease,Galantamine,CbGcGbCpD,0,0,2,0.001874,0.002421,0.000175
2,Alzheimer's disease,Galantamine,CbGcGcGdD,0,0,2,0.022984,0.022924,0.029113
3,Alzheimer's disease,Galantamine,CbGcGiGdD,1,0,1,0.013181,0.000000,0.100678
4,Alzheimer's disease,Galantamine,CbGcGr>GuD,1,0,1,0.029613,0.000000,0.135766


In [19]:
# Percent of the time that each complexity rank (higher is harder) was fastest
query_summary_df.rank_of_fastest.value_counts(normalize=True)

1    0.3723
2    0.2432
3    0.1855
4    0.1317
5    0.0673
Name: rank_of_fastest, dtype: float64

In [20]:
# Mean of each column: Percentages for binary columns
query_summary_df.mean()

optimal_is_best      0.372200
midpoint_is_best     0.408500
rank_of_fastest      2.278500
min_seconds          0.617707
delay_of_optimal     0.057719
delay_of_midpoint    0.011089
dtype: float64

In [21]:
metapath_summary_df = query_summary_df.groupby('metapath').mean().reset_index()
metapath_summary_df.head()

,metapath,optimal_is_best,midpoint_is_best,rank_of_fastest,min_seconds,delay_of_optimal,delay_of_midpoint
0,CbG<rGr>GaD,0.025,0.870,2.125,0.151339,1.096688,0.001552
1,CbGcGbCpD,0.155,0.260,2.640,0.001927,0.001558,0.000375
2,CbGcGcGdD,0.075,0.325,2.490,0.008136,0.022918,0.013493
3,CbGcGiGdD,0.490,0.235,2.255,0.005744,0.001127,0.034586
4,CbGcGr>GuD,0.545,0.275,2.015,0.007687,0.001011,0.063892


In [23]:
def summary(df):
    row = pandas.Series()
    best_time = df.seconds.min()
    row['fastest_index'] = df.join_index.iloc[numpy.argmin(list(df.seconds))]
    row['delay_of_optimal'] = df.loc[df.optimal_index, 'seconds'].iloc[0] - best_time
    row['delay_of_midpoint'] = df.loc[df.midpoint_index, 'seconds'].iloc[0] - best_time
    return row

metapath_choice_df = complexity_df.groupby(
    ['metapath', 'join_index', 'optimal_index', 'midpoint_index']
).seconds.mean().reset_index().groupby('metapath').apply(summary).reset_index()
metapath_choice_df.fastest_index = metapath_choice_df.fastest_index.astype(int)
metapath_choice_df.head()

,metapath,fastest_index,delay_of_optimal,delay_of_midpoint
0,CbG<rGr>GaD,2,1.095136,0.000000
1,CbGcGbCpD,2,0.001184,0.000000
2,CbGcGcGdD,3,0.020069,0.010644
3,CbGcGiGdD,3,0.000000,0.033459
4,CbGcGr>GuD,3,0.000000,0.062881


In [27]:
query_summary_df.to_csv('data/query-summaries.tsv', sep='\t', index=False, float_format='%.4g')
metapath_summary_df.to_csv('data/metapath-summaries.tsv', sep='\t', index=False, float_format='%.4g')
metapath_choice_df.to_csv('data/index-choice-by-metapath.tsv', sep='\t', index=False, float_format='%.4g')

In [32]:
# import seaborn
# %matplotlib inline
# ax = seaborn.lmplot(x='complexity', y='log_seconds', data=complexity_df, lowess=True, row='metapath', hue='join_index')